# Select database (i.e. staging, production, local authenticated, local unauthenticated)

In [1]:
import requests
import os
import obi_one as obi
virtual_lab_id=obi.LAB_ID_STAGING_TEST
project_id=obi.PROJECT_ID_STAGING_TEST
from entitysdk import Client, ProjectContext

from obi_auth import get_token
from obi_notebook import get_projects

obi_one_api_url = "http://127.0.0.1:8100"

token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=obi.LAB_ID_STAGING_TEST, project_id=obi.PROJECT_ID_STAGING_TEST)
db_client = Client(api_url="https://staging.openbraininstitute.org/api/entitycore", project_context=project_context, token_manager=token)

## Test the mesh validate endpoint

In [ ]:
params = {}
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {token}' if token else None,
}

UPLOAD_ENDPOINT= f"{obi_one_api_url}/declared/validate-mesh-file"

"""Test uploading a valid .obj file."""
print("\nTesting valid file upload (.obj)...")

temp_file_path = "../../../tests/data/cube.obj"

try:
    with open(temp_file_path, 'rb') as file:
        files = {'file': (os.path.basename(temp_file_path), file, 'text/plain')}
        response = requests.post(UPLOAD_ENDPOINT, files=files, headers=headers, params=params)
    print(f"Status Code: {response.status_code}")
except Exception as e:
    print(f"Error: {str(e)}")

## Test the electrophysiology endpoint

In [ ]:
# Construct the full endpoint URL
url = f"{obi_one_api_url}/declared/validate-electrophysiology-protocol-nwb-file"

params = {}
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

temp_file_path = "../../../tests/data/S1FL_L5_DBC_cIR_4.nwb"
try:
    with open(temp_file_path, 'rb') as file:
        files = {'file': (os.path.basename(temp_file_path), file, 'text/plain')}
        response = requests.post(url, files=files, headers=headers, params=params)
        print(f"Status Code: {response.status_code}")
        print(f"Response: {response.json()}")
except Exception as e:
    print(f"Error: {str(e)}")

# Fetch a morphology

In [ ]:
from entitysdk.models.cell_morphology import (
    CellMorphology,
)
kwargs = {"name": "ch150801A1"}
cell_morphologies = db_client.search_entity(
    entity_type=CellMorphology, query=kwargs, limit=1
).one()

cell_morphology_id = cell_morphologies.id
print(cell_morphology_id)

# Call the declared morphology metrics endpoint

In [ ]:
# Construct the full endpoint URL
url = f"{obi_one_api_url}/declared/neuron-morphology-metrics/{cell_morphology_id}"

# Set headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Make the GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")


# Call the generated morphology metrics endpoint

In [ ]:
import requests

# Construct the full endpoint URL
# url = f"{obi_one_api_url}/generated/morphology-metrics-scan-config-run-grid"
url = f"{obi_one_api_url}/declared/scan_config/grid-scan-coordinate-count"

# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request body — adjust this with the actual morphology metrics form data!
# Placeholder example:
request_body = {
  "type": "MorphologyMetricsScanConfig",
  "initialize": {
    "type": "MorphologyMetricsScanConfig.Initialize",
    "morphology": {
      "type": "CellMorphologyFromID",
      "id_str": str(cell_morphology_id),
    }
  }
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

# Fetch an electrophysiology recording

In [ ]:
from entitysdk.models.electrical_cell_recording import (
    ElectricalCellRecording,
)
kwargs = {"name": "C190101A1-MT-C1"}
Electrical_cell_recording = db_client.search_entity(
    entity_type=ElectricalCellRecording, query=kwargs, limit=1
).one()

Electrical_cell_recording_id = str(Electrical_cell_recording.id)
print(Electrical_cell_recording_id)

# Call the declared electrophysiology recording metrics endpoint

In [ ]:
import requests

trace_id = str(Electrical_cell_recording_id)
url = f"{obi_one_api_url}/declared/electrophysiologyrecording-metrics/{trace_id}"

# Define query parameters
params = {
    "requested_metrics": ["AP_height", "spike_count"],
    "protocols": "idrest",
    "min_value": 0.1,
    "max_value": 0.4,
}

headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
}

if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

# Fetch a circuit

In [ ]:
# .one() circuit in staging temporarily missing node_sets.json
# from entitysdk.models.circuit import (
#     Circuit,
# )
# kwargs = {}
# circuit = db_client.search_entity(
#     entity_type=Circuit, query=kwargs, limit=1
# ).one()

# circuit_id = circuit.id
# print(circuit_id)


circuit_id = str("a581a7d9-435e-4aa6-8f27-b0fcb13ec66c")

# Call the circuit metrics endpoint

In [ ]:
import requests

circuit_id = str(circuit_id)
url = f"{obi_one_api_url}/declared/circuit-metrics/{circuit_id}"

params = {}

headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
}

if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
import requests

url = f"{obi_one_api_url}/declared/mapped-circuit-properties/{circuit_id}"

params = {}

headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
}

if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
import requests

# Construct the full endpoint URL
url = f"{obi_one_api_url}/declared/parametric-multi-value"

# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# request_body = {
#     "type": "IntRange",
#     "start": -4,
#     "step": 1,
#     "end": 11,
# }
request_body = {
    "type": "FloatRange",
    "start": -0.3,
    "step": 0.1,
    "end": -0.11,
}

query_params = {
    "ge": -0.2,
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body, params=query_params)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")